## Mathematik für Biologiestudierende II

Sommersemester 2024

25.06.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
sns.set_theme()
import warnings
warnings.filterwarnings('ignore', message='The figure layout has changed')

# Lineare Modelle

#### Themen heute

* mehrere erklärende Variablen
* Transformationen
* Normalverteilungsannahmen
* kategorielle erklärende Variablen

# Lineare Modelle mit kategoriellen Daten

Wir kommen zu dem Rattenbeispiel aus Lektion 21 zurück:

* kontaminiertes Gelände: fange 10 Ratten
* unbelastetes Vergleichsgelände:  fange 10 Ratten
* für jede Ratte wird ihr Alter in Monaten und der Bleigehalt im Gewebe bestimmt

In [ ]:
df = pd.read_csv('ratten.csv')

In [ ]:
sns.lmplot(df, x='Alter', y='Belastung', hue='Gelände');

* Der t-Test zeigte keinen Unterschied zwischen den Ratten auf kontaminierten und nicht kontaminiertem Gelände.
* Die Ratten auf dem kontaminierten Gelände sind aber im Schnitt jünger.  
* Wir wollen gleichaltrige Ratten vergleichen

In [ ]:
formel = 'Belastung ~ Alter + Gelände'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

* `Gelände[T.unbelastet]` ist signifikant
* Allerdings ist das der Unterschied bei Alter = 0
* Das ist Unsinn

* Wir vergleichen im Alter von 8 und 9 Monaten

In [ ]:
anfrage = pd.DataFrame()
anfrage['Alter'] = [8,8,9,9]
anfrage['Gelände'] = ['belastet', 'unbelastet', 'belastet', 'unbelastet']
anfrage

In [ ]:
res.get_prediction(anfrage).summary_frame()

* Relevant sind hier die Konfidenzintervalle für die Mittelwerte
* Für beide Werte von `Alter` ist die untere Vertrauensgrenze für die Belastung im Gewebe der Ratten vom kontaminierten Gelände höher als die obere Vertrauensgrenze für die Ratten vom unkontaminierten Gelände
* Zum Signifikanzniveau $\alpha = 0.95$ ist der Unterschied in der Bleibelastung signifikant

## Bestimmung des p-Werts

* Der p-Wert wird nur für den Unterschies in 0 ausgegeben.
* Trick:  Verlegung des Nullpunkts.

* Im Beispiel verlegen wird den Nullpunkt auf 8 Monate.
* Wir führen in der Tabelle also eine Spalte mit der Altersdifferenz zu 8 Monaten ein

In [ ]:
df['Altersdifferenz'] = df.Alter - 8
df

In [ ]:
formel2 = 'Belastung ~ Altersdifferenz + Gelände'
modell2 = smf.ols(formel2, df)
res2 = modell2.fit()

In [ ]:
res2.summary()

* Der p-Wert ist 0.002

# ANOVA als lineares Modell

### Beispiel Schadstoffkonzentration

* An fünf verschiedenen Messstellen wurde die Konzentration eines Schadstoffs gemessen
* Hat die Messstelle einen Einfluss auf die Konzentration?
* Die Messstelle ist der Faktor
* Die Konzentration ist die Zielvariable

In [ ]:
u_schad = "https://www.math.uni-duesseldorf.de/~braun/bio2324/data/schadstoffe.csv"
df = pd.read_csv(u_schad, index_col=0)
df

In [ ]:
g1 = df[df.Messstelle==1].Konzentration
g2 = df[df.Messstelle==2].Konzentration
g3 = df[df.Messstelle==3].Konzentration
g4 = df[df.Messstelle==4].Konzentration
g5 = df[df.Messstelle==5].Konzentration

In [ ]:
res = stats.f_oneway(g1, g2, g3, g4, g5)
res

Das ist ein lineares Modell, bei dem die einzige erklärende Variable kategoriell ist

In [ ]:
formel = 'Konzentration ~ Messstelle'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

* Sehen Sie, wie das schief gegangen ist?

In [ ]:
formel = 'Konzentration ~ C(Messstelle)'
modell = smf.ols(formel, df)
res = modell.fit()

* durch `C(Messstelle)` wird festgelegt, dass es sich um einen kategoriellen Wert handelt
* wenn die Messstellen nicht durch Nummern, sondern durch Text (z.B. "unterer Bachlauf") bezeichnet sind, ist das nicht nötig

In [ ]:
res.summary()

* Der Eintrag `Prob (F-statistic)` zeigt den p-Wert der ANOVA
* Der Eintrag `F-statistic` teigt die Teststatistik der ANOVA

* Die Einträge `C(Messstelle)[T.2]` usw. zeigen **nicht** die post-Hoc Analyse

In [ ]:
stats.ttest_ind(g1, g3)

In [ ]:
stats.ttest_ind?

In [ ]:
g1.describe()

In [ ]:
import statsmodels.api as sm

In [ ]:
sm.stats.anova_lm(res)

# Regression im exponentiellen Modell

## Beispiel Covid-Erkrankungen

In [ ]:
df = pd.read_csv('corona.csv')
df.head()

In [ ]:
df.columns

In [ ]:
ax = sns.scatterplot(data=df, x="Tag (im März)", y="Anzahl");

* Wir hatten damals starke Schankungen durch unterschiedliche Verzögerungen der Berichte der Gesundheitsämter
* Im  großen und ganzen war das Wachstum aber exponentiell
* Wir skalieren die y-Achse logarithmisch

In [ ]:
ax.set_yscale('log')
ax.get_figure()

* Wir logarithmieren die Anzahlen
* Wir machen lineare Regression für die logarithmierten Anzahlen
* Wir exponentieren wieder

In [ ]:
df['logAnzahl'] = np.log(df.Anzahl)
df['Tag'] = df['Tag (im März)']

In [ ]:
formel = 'logAnzahl ~ Tag'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

In [ ]:
tage = np.arange(3, 24)
gerade = 0.226*tage + 3.441
expKurve = np.exp(gerade)
sns.lineplot(x=tage, y=expKurve)

In [ ]:
sns.lineplot(x=tage, y=expKurve, ax=ax)
ax.get_figure()

In [ ]:
ax.set_yscale('linear')
ax.get_figure()